In [ ]:
import pandas as pd 
import numpy as np
import numba as nb
from components import scraping, modeling
# key = modeling.keywordExtractor(dir='data/preprocess/eng_han.csv')
keyword = pd.read_parquet('data/update/extracted_keywords.parquet')
book_info = pd.read_parquet('data/local_data/backup/book_info.parquet')

def add_None(lst) :
    # pad도 notion에 넣기 
    pad_length=20-len(lst)
    return np.pad(np.array(lst),(0,pad_length),'constant', constant_values=('None'))

book_keyword = np.row_stack(list(map(add_None,keyword.keyword.values)))
isbn_list = keyword['isbn13'].values

In [19]:
from configs import Deployment
lib_book= pd.read_parquet('data/update/lib_books.parquet')
lib_dict = Deployment().lib_codes
lib_book['lib_code'] = lib_book['lib_code'].apply(lambda x : lib_dict[x])

In [20]:
from gensim.models import keyedvectors

model = keyedvectors.load_word2vec_format('data/update/w2v')

### Search algorithm

In [21]:
from typing import * 
user_search = ['파이썬']

def extract_recommand_book_isbn(user_search:List[str]) -> List[str] : 
    # extract recommand_keyword
    recommand_keyword = model.most_similar(positive=user_search,topn=15)
    np_recommand_keyword = np.array(list(map(lambda x : x[0],recommand_keyword)))

    # calculate top_k_idx
    user_point = np.isin(book_keyword,np.array(user_search)).sum(axis=1)
    recommand_point = np.isin(book_keyword,np_recommand_keyword).sum(axis=1)
    total_point = (user_point * 3) + recommand_point
    top_k_idx = np.argsort(total_point)[::-1][:50]

    return isbn_list[top_k_idx]

# 281us
# result = book_info.iloc[pd.Index(book_info['ISBN']).get_indexer(sorted_isbn)]

In [22]:
extract_recommand_book_isbn(user_search)

array(['9791162241905', '9791161756233', '9788957175231', '9791157832361',
       '9791162242414', '9791191198010', '9791161753324', '9791165366407',
       '9791197347023', '9791192373126', '9788968480478', '9791158391225',
       '9791162245569', '9791185578798', '9788970505220', '9791190283700',
       '9791158391881', '9791197888014', '9791196440923', '9791158393571',
       '9791158392260', '9791161340210', '9791161750910', '9788970505558',
       '9791161752136', '9791162244210', '9791127453497', '9788960778535',
       '9791160507812', '9791192038049', '9791162245545', '9791156006978',
       '9791165211639', '9791158392284', '9791186710678', '9791160505979',
       '9788970938424', '9791155505649', '9791190014281', '9791160503371',
       '9791185578729', '9791158392390', '9791158391836', '9788956748573',
       '9788931555684', '9791165920722', '9791162245415', '9791158391126',
       '9791158391522', '9788970509402'], dtype=object)

In [ ]:
# a(36.9ms) => b(22.1ms)
a = (np_recommand_keyword==book_keyword[...,None]).any(axis=1).sum(axis=1)
b = np.isin(book_keyword,np_recommand_keyword).sum(axis=1)